## Appendix Table A4: Latinx First-Year Enrollments with Admissions Control

from White Flight in Higher Education? The Case of Hispanic-Serving Institutions

by Laura Hamilton, Charlie Eaton, and Simon Cheng

In [1]:
quietly {

use data/hdef19902019, clear
keep if ftfeftotlt>100 & ftfeftotlt<.
    
keep if admitpct!=.
    gen pubpriv="public"
    replace pubpriv="private" if control==2
    
foreach var in efbkaat efhispt efwhitt efaiant efasiat {
quietly    gen pctt`var'= t`var' / teftotlt * 100
quietly gen lnftf`var'=log(ftf`var')
    quietly gen lnt`var'=log(t`var')
}
foreach var in cpopam cpopas cpopbl cpopwh cpophi populationam populationas populationbl populationhi populationwh {
        gen ln`var'=log(`var')
    }

quietly gen lnftfeftotlt=log(ftfeftotlt)

quietly gen hispserve=0 if pcttefhispt!=.
quietly replace hispserve=1 if pcttefhispt>=25 & pcttefhispt<.

xtset unitid year

**create var for year in which event occurs observation year of event for subject**
by unitid: gen transition=year if hispserv==1 & l.hispserv==0

**create var for year in which event occurs in all observations for all years for subject**
by unitid: egen hispserveyear = min(transition)

**generate a var for the number of years relative to the year of the event (e.g. + or - 2 years)

gen hispserveyeardiff= year-hispserveyear
    
**bin up year differences larger than absolute value of 10**
*replace hispserveyeardiff =10 if hispserveyeardiff <9999 & hispserveyeardiff >10
*replace hispserveyeardiff =-10 if hispserveyeardiff <-10
    
    gen afteryrs=0
replace afteryrs=hispserveyeardiff if hispserveyeardiff>0 & hispserveyeardiff<.
gen beforeyrs=0
replace beforeyrs=hispserveyeardiff if hispserveyeardiff<0
gen beforeafteryrs=0
replace beforeafteryrs=hispserveyeardiff if hispserveyeardiff!=.
    
gen switchers = (hispserveyeardiff<.)
    label var beforeyrs "before years"
    label var afteryrs "after years"
    label var beforeafteryrs "before and after years"
    label var lnftfefwhitt "white"
    label var lnftfefasiat "Asian"
    label var lnftfefbkaat "Black"
    label var admitpct "% admitted"
}
sum beforeyrs afteryrs beforeafteryrs



. sum beforeyrs afteryrs beforeafteryrs

    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
   beforeyrs |     25,807   -.3712946    1.896903        -18          0
    afteryrs |     25,807    .1091177    .8945122          0         17
beforeafte~s |     25,807   -.2621769    2.116466        -18         17


In [2]:
*** Check and clear xt

    qui xtset
    qui set line 105
    qui set more off

foreach pubpriv in public private {

est clear
        qui eststo SwPost: reghdfe lnftfefhispt beforeyrs afteryrs lnftfeftotlt admitpct ///
        lnpop* lncpop* i.year if switchers==1 & pubpriv=="`pubpriv'", absorb(unitid) cluster(unitid)

        qui eststo counterPost: reghdfe lnftfefhispt beforeyrs afteryrs lnftfeftotlt admitpct ///
        lnpop* lncpop* i.year if pcttefhispt>=15 & pubpriv=="`pubpriv'", absorb(unitid) cluster(unitid)
        
        qui eststo NSwPost: reghdfe lnftfefhispt beforeyrs afteryrs lnftfeftotlt admitpct ///
        lnpop* lncpop* i.year if pubpriv=="`pubpriv'", absorb(unitid) cluster(unitid)
        
        qui eststo Swdiff: reghdfe lnftfefhispt afteryrs beforeafteryrs lnftfeftotlt admitpct ///
        lnpop* lncpop* i.year if switchers==1 & pubpriv=="`pubpriv'", absorb(unitid) cluster(unitid)

        qui eststo counterdiff: reghdfe lnftfefhispt afteryrs beforeafteryrs lnftfeftotlt admitpct ///
        lnpop* lncpop* i.year if pcttefhispt>15 & pubpriv=="`pubpriv'", absorb(unitid) cluster(unitid)
        
        qui eststo NSwdiff: reghdfe lnftfefhispt afteryrs beforeafteryrs lnftfeftotlt admitpct ///
        lnpop* lncpop* i.year if pubpriv=="`pubpriv'", absorb(unitid) cluster(unitid)
        
        esttab SwPost counterPost NSwPost Swdiff counterdiff NSwdiff, star(* .05 ** .01 *** .001) ///
        stats(r2_a_within N_clust N, fmt(2 %9.0fc %9.0fc) label("r2" "schools" "N")) ///
        keep(beforeyrs afteryrs beforeafteryrs admitpct) se(%9.3fc) b(%9.3fc) ///        
        order(beforeyrs afteryrs beforeafteryrs admitpct) ///
        label mtitle("Switchers" "Counterfac" "All" "Switchers" "Counterfac ""All") ///
        prehead("`pubpriv' Latinx enrollment") nonotes modelwidth(10)
            
   qui esttab SwPost counterPost NSwPost Swdiff counterdiff NSwdiff using tables/ta4_latinx`pubpriv'admit.rtf, star(* .05 ** .01 *** .001) ///
        stats(r2_a_within N_clust N, fmt(2 %9.0fc %9.0fc) label("r2" "schools" "N")) ///
        keep(beforeyrs afteryrs beforeafteryrs admitpct) se(%9.3fc) b(%9.3fc) ///
        order(beforeyrs afteryrs beforeafteryrs admitpct) ///
        label mtitle("Switchers" "Counter- factual" "All" "Switchers" "Counter- factual ""All") replace ///
        title("\b `pubpriv' Latinx enrollment") nonotes onecell compress
}






public Latinx enrollment
                            (1)           (2)           (3)           (4)           (5)           (6)   
                      Switchers    Counterfac           All     Switchers    Counterfac           All   
--------------------------------------------------------------------------------------------------------
before years              0.036***      0.018***     -0.015***                                          
                        (0.008)       (0.004)       (0.004)                                             

after years               0.015*        0.012**      -0.022***     -0.021**      -0.005        -0.007   
                        (0.007)       (0.004)       (0.005)       (0.006)       (0.006)       (0.006)   

before and after y~s                                                0.036***      0.018***     -0.015***
                                                                  (0.008)       (0.004)       (0.004)   

% admitted            